# VacationPy
----

#### Note
* This notenook utilizes Google API. Keep an eye on API utilization. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.


In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Getting Weather Data from WeatherPy results into DataFrame

In [17]:
# Getting weather data from WeatherPy work and storing in a dataframe
weather_data_file = "../output_data/cities.csv"
weather_data = pd.read_csv(weather_data_file)
# Displaying preview of weather data
weather_data.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,Ribeira Grande,38.52,-28.70,60.8,67,40,6.93,PT,1603262680
1,1,Grindavik,63.84,-22.43,37.4,86,90,9.17,IS,1603262680
2,2,Christchurch,-43.53,172.63,59.0,67,0,8.05,NZ,1603262494
3,3,Sedalia,38.70,-93.23,46.4,93,1,8.05,US,1603262680
4,4,La Ciotat,43.17,5.60,61.0,79,53,5.01,FR,1603262681


### Humidity Heatmap

In [22]:
# Configuring gmaps with API key
gmaps.configure(api_key=g_key)

# Defining parameters to be passed to gmaps api
locations = weather_data[["Lat", "Lon"]].astype(float)
humidity = weather_data["Humidity"].astype(float)
max_humidity = weather_data["Humidity"].max()

In [20]:
# Configuring gmaps layout
fig = gmaps.figure(layout={
        'width': '960px',
        'height': '600px',
        'padding': '3px'
    })

# Creating a Humidity Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=True, max_intensity=max_humidity)

# Adding Heatmap layer to the map
fig.add_layer(heat_layer)

# Displaying the map
fig

Figure(layout=FigureLayout(height='600px', padding='3px', width='960px'))

### Selection of Vacation Cities
#### Criteria for selection of vacation ciity
* A max temp < 80 degree and > 70 degree
* Wind speed < 10 mph
* Zero cloudiness


In [6]:
# Storing vacation city data in a dataframe with above listed criteria
vacation_data = weather_data[(weather_data["Max Temp"]<80.00)
                            & (weather_data["Max Temp"]>70.00)
                            & (weather_data["Windspeed"]<10.00)
                            & (weather_data["Cloudiness"]==0)]

# Removing NA values from the dataframe
vacation_data = vacation_data.dropna()

# Displaying the final list of Vacation City list
vacation_data

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
40,40,Adrar,20.50,-10.07,79.09,20,0,4.92,MR,1603262687
57,57,Buraidah,26.33,43.98,75.20,29,0,2.24,SA,1603262689
103,103,Mubi,10.27,13.26,78.89,40,0,1.32,NG,1603262697
110,110,Turaif,31.67,38.66,75.20,14,0,9.17,SA,1603262698
116,116,Cabo San Lucas,22.89,-109.91,79.00,69,0,1.01,MX,1603262460
125,125,Léo,11.10,-2.10,76.89,70,0,5.35,BF,1603262700
169,169,Kombissiri,12.07,-1.34,75.20,83,0,2.24,BF,1603262710
197,197,Jumla,29.27,82.18,70.27,20,0,5.17,NP,1603262715
243,243,Rajaori,33.38,74.30,78.96,20,0,6.62,IN,1603262490
253,253,Najrān,17.49,44.13,77.00,17,0,2.24,SA,1603262726


### Hotel Map

In [7]:
# Finding the closest hotel of each type to coordinates

# Setting up baseurl for api call and radius for searching Hotel for Vacation City
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000

# Running a loop through all cities in Vacation City list to find Hotels
for index, row in vacation_data.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]
    city = row["City"]
   
    params = {
        "location": f"{lat},{lon}",
        "radius": radius,
        "type": "lodging",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    
    # Handling exceptions in case no Hotel found for a City and adding Hotel Name in Vacation Dataframe
    try:
        # Getting first hotel name from the list of hotels received from api call
        hotel = results[0]["name"]
        print(f"Closest Hotel to city {city} is {hotel}.")
        vacation_data.loc[index, "Hotel Name"] = hotel
        
    except (KeyError, IndexError):
        print("Could not find a nearby hotel...skipping")

Could not find a nearby hotel...skipping
Closest Hotel to city Buraidah is Mövenpick Hotel Qassim.
Closest Hotel to city Mubi is Hotel.
Closest Hotel to city Turaif is Swiss Spirit Hotel & Suites Turaif.
Closest Hotel to city Cabo San Lucas is Hotel Tesoro Los Cabos.
Closest Hotel to city Léo is Hotel Sissilis.
Closest Hotel to city Kombissiri is Mitibkieta Annexe.
Closest Hotel to city Jumla is Hotel Sundar Rara Karnali.
Closest Hotel to city Rajaori is Hotel Ashoka.
Closest Hotel to city Najrān is محمد هادي ال فطيح.
Closest Hotel to city Sakakah is Fakhamat Al Orjoana Apartment.
Closest Hotel to city Dalbandin is Police station anti NARCOTICS FORCE.
Closest Hotel to city Sarakhs is Sarakhs Abrisham Hotel.
Closest Hotel to city Panabá is Hotel Amor.
Closest Hotel to city Presidencia Roque Sáenz Peña is Hotel Aconcagua.
Closest Hotel to city Rāvar is computertik.
Closest Hotel to city Biu is EYN LCC MANDARAGIRAU.
Closest Hotel to city Byron Bay is Backpackers Inn on the Beach.
Closest 

In [8]:
# Removing cities from Vacation Cities, where no hotel has been found
hotel_df = vacation_data.dropna()

# Displaying final list of Vacation city with Hotel Name
hotel_df

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
57,57,Buraidah,26.33,43.98,75.20,29,0,2.24,SA,1603262689,Mövenpick Hotel Qassim
103,103,Mubi,10.27,13.26,78.89,40,0,1.32,NG,1603262697,Hotel
110,110,Turaif,31.67,38.66,75.20,14,0,9.17,SA,1603262698,Swiss Spirit Hotel & Suites Turaif
116,116,Cabo San Lucas,22.89,-109.91,79.00,69,0,1.01,MX,1603262460,Hotel Tesoro Los Cabos
125,125,Léo,11.10,-2.10,76.89,70,0,5.35,BF,1603262700,Hotel Sissilis
169,169,Kombissiri,12.07,-1.34,75.20,83,0,2.24,BF,1603262710,Mitibkieta Annexe
197,197,Jumla,29.27,82.18,70.27,20,0,5.17,NP,1603262715,Hotel Sundar Rara Karnali
243,243,Rajaori,33.38,74.30,78.96,20,0,6.62,IN,1603262490,Hotel Ashoka
253,253,Najrān,17.49,44.13,77.00,17,0,2.24,SA,1603262726,محمد هادي ال فطيح
289,289,Sakakah,29.97,40.21,75.20,13,0,7.52,SA,1603262733,Fakhamat Al Orjoana Apartment


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [15]:
# Configuring marker layer for display with Heatmap layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Adding marker layer to existing heatmap figure
fig.add_layer(marker_layer)

# Displaying updated figure
fig


Figure(layout=FigureLayout(height='600px', padding='3px', width='960px'))